In [2]:
import torch

device = torch.device("cuda")

In [3]:
from spuco.utils import set_seed

set_seed(0)

In [1]:
from spuco.robust_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE
root = "data/mnist/"

trainset = SpuCoMNIST(
    root=root,
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.95,
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root=root,
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


/home/yeutong/miniconda3/envs/spuco/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 10000/10000 [00:00<00:00, 10163.40it/s]


In [4]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root=root,
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


100%|██████████| 11996/11996 [00:01<00:00, 10213.43it/s]


In [5]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from spuco.robust_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T
import numpy as np

In [7]:
train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
not_shuffle_train_loader = DataLoader(trainset, batch_size=64, shuffle=False)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)

In [8]:
from spuco.models import model_factory

# model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

from spuco.evaluate import Evaluator

# Initialize the model and optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=False)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
num_epochs = 10
# train_predictions_history = [[] for _ in range(len(trainset))]
# test_predictions_history = [[] for _ in range(len(testset))]
train_acc_history_lst = []
test_acc_history_lst = []

# Training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Evaluation function
def evaluate(model, data_loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    acc_history = []
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            acc_history.extend((predicted == target).cpu().numpy())
            predictions.extend(predicted.cpu().numpy())
    return correct / total, acc_history

# Calculate forgetability scores
def calculate_forgetability(predictions_history):
    forgetability_scores = []
    for sample_predictions in predictions_history:
        correct_to_incorrect = 0
        for i in range(1, len(sample_predictions)):
            if sample_predictions[i-1] == 1 and sample_predictions[i] == 0:
                correct_to_incorrect += 1

        if sum(sample_predictions) == 0:
            correct_to_incorrect = (num_epochs // 2) + 1
        forgetability_scores.append(correct_to_incorrect)
    return forgetability_scores



for epoch in range(num_epochs):
    train(model, train_loader, optimizer, criterion, device)
    train_acc, train_acc_history = evaluate(model, not_shuffle_train_loader, criterion, device)
    test_acc, test_acc_history = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

    train_acc_history_lst.append(train_acc_history)
    test_acc_history_lst.append(test_acc_history)



Epoch 1/10, Train Acc: 0.9503, Test Acc: 0.2230
Epoch 2/10, Train Acc: 0.9513, Test Acc: 0.2658
Epoch 3/10, Train Acc: 0.9541, Test Acc: 0.2900
Epoch 4/10, Train Acc: 0.9560, Test Acc: 0.3162
Epoch 5/10, Train Acc: 0.9558, Test Acc: 0.3123
Epoch 6/10, Train Acc: 0.9603, Test Acc: 0.3962
Epoch 7/10, Train Acc: 0.9611, Test Acc: 0.3883
Epoch 8/10, Train Acc: 0.9643, Test Acc: 0.4568
Epoch 9/10, Train Acc: 0.9617, Test Acc: 0.3965
Epoch 10/10, Train Acc: 0.9656, Test Acc: 0.4737


In [9]:
train_acc_history_lst = np.array(train_acc_history_lst).T
train_forgetability_scores = calculate_forgetability(train_acc_history_lst)


In [10]:
# group accuracy for the last epoch
print("Trainset accuracy for each group")
for group, members in trainset.group_partition.items():
    print(group, end=' ')
    for epoch in range(10):
        group_score = train_acc_history_lst[members, epoch]
        print(f"{group_score.mean():.4f}", end=' ')
    print()

Trainset accuracy for each group
(2, 2) 1.0000 0.9999 0.9991 0.9987 0.9995 0.9975 0.9996 0.9981 0.9985 0.9970 
(0, 0) 0.9994 0.9886 0.9994 0.9996 0.9978 0.9983 0.9986 0.9969 0.9984 0.9981 
(4, 4) 1.0000 1.0000 1.0000 0.9990 1.0000 1.0000 0.9997 0.9992 0.9996 0.9997 
(1, 1) 1.0000 1.0000 0.9997 1.0000 0.9997 1.0000 0.9995 0.9995 0.9998 0.9977 
(3, 3) 1.0000 1.0000 0.9975 0.9990 0.9972 0.9948 0.9981 0.9956 0.9995 0.9963 
(3, 1) 0.0000 0.0000 0.0551 0.0000 0.0787 0.0000 0.1181 0.1575 0.1024 0.2835 
(0, 2) 0.0000 0.0000 0.1440 0.2800 0.1360 0.5440 0.1520 0.5520 0.3760 0.6960 
(1, 0) 0.0896 0.6940 0.2015 0.3358 0.2239 0.5522 0.5000 0.5448 0.5746 0.6567 
(0, 4) 0.0000 0.0000 0.0667 0.3259 0.0444 0.1333 0.3333 0.5704 0.3630 0.5185 
(2, 1) 0.0000 0.0278 0.1481 0.0185 0.0648 0.0463 0.2778 0.2870 0.2778 0.3796 
(2, 3) 0.0000 0.0000 0.0943 0.0000 0.0094 0.2736 0.2453 0.1887 0.0943 0.0943 
(3, 4) 0.0000 0.0000 0.0000 0.3590 0.0000 0.0000 0.1966 0.1453 0.1368 0.2222 
(1, 3) 0.0000 0.0000 0.2059 0.2

In [11]:
print("Trainset forgetability score each group")
train_forgetability_scores = np.array(train_forgetability_scores)
count = 0
for group, members in trainset.group_partition.items():
    group_score = train_forgetability_scores[members]
    print(group, group_score.mean())


Trainset forgetability score each group
(2, 2) 0.007486255702421336
(0, 0) 0.017151767151767153
(4, 4) 0.0018983807928531547
(1, 1) 0.0034854591003158695
(3, 3) 0.013187763485028646
(3, 1) 4.503937007874016
(0, 2) 2.488
(1, 0) 2.246268656716418
(0, 4) 3.066666666666667
(2, 1) 3.787037037037037
(2, 3) 4.433962264150943
(3, 4) 4.282051282051282
(1, 3) 3.4705882352941178
(0, 1) 2.5213675213675213
(4, 2) 6.0
(4, 0) 2.037313432835821
(2, 0) 3.0172413793103448
(4, 1) 5.345132743362832
(0, 3) 1.7050359712230216
(4, 3) 3.8547008547008548
(3, 0) 1.8032786885245902
(2, 4) 6.0
(3, 2) 3.89922480620155
(1, 4) 5.015625
(1, 2) 3.3854166666666665


In [62]:
train_forgetability_scores.shape

(48004,)

In [63]:
train_acc_history_lst[:, -1].shape

(48004,)

In [13]:
# group correct into (0,0), and incorrect into (1,1)

jtt_group_partition = {(0,0): [], (0,1): []}
for i in range(train_acc_history_lst.shape[0]):
    if train_acc_history_lst[i, -1]: # correct
        jtt_group_partition[(0,0)].append(i)
    else: # incorrect
        jtt_group_partition[(0,1)].append(i)

In [15]:
from collections import defaultdict

# fgt_group_partition = {(0, i): [] for i in range(7)}
fgt_group_partition = defaultdict(list)
for i in range(train_forgetability_scores.shape[0]):
    if train_forgetability_scores[i] >= 1: # forgettable
        fgt_group_partition[(0,1)].append(i)
    else: # not forgettable
        fgt_group_partition[(0,0)].append(i)

    # group = (0, train_forgetability_scores[i].item())
    # if group not in fgt_group_partition:
    #     fgt_group_partition[group] = []
    # fgt_group_partition[group].append(i)

In [16]:
import copy

# clone model
model_jtt = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)
model_fgt = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

model_jtt.load_state_dict(copy.deepcopy(model.state_dict()))
model_fgt.load_state_dict(copy.deepcopy(model.state_dict()))


<All keys matched successfully>

In [17]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model_jtt,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   0%|          | 0/25 [00:00<?, ?it/s]

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:08,  2.99it/s]

Group (0, 0) Accuracy: 99.76359338061465


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:07,  3.00it/s]

Group (0, 1) Accuracy: 53.90070921985816


...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:08<00:00,  2.75it/s]

Group (4, 3) Accuracy: 24.494949494949495


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:08<00:00,  2.84it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 99.76359338061465,
 (0, 1): 53.90070921985816,
 (0, 2): 66.19385342789599,
 (0, 3): 78.01418439716312,
 (0, 4): 59.1016548463357,
 (1, 0): 58.924205378973106,
 (1, 1): 100.0,
 (1, 2): 23.529411764705884,
 (1, 3): 34.068627450980394,
 (1, 4): 7.598039215686274,
 (2, 0): 33.86666666666667,
 (2, 1): 30.666666666666668,
 (2, 2): 99.46666666666667,
 (2, 3): 9.066666666666666,
 (2, 4): 0.0,
 (3, 0): 62.311557788944725,
 (3, 1): 22.670025188916878,
 (3, 2): 22.418136020151135,
 (3, 3): 99.24433249370277,
 (3, 4): 14.6095717884131,
 (4, 0): 65.4911838790932,
 (4, 1): 7.5566750629722925,
 (4, 2): 0.0,
 (4, 3): 24.494949494949495,
 (4, 4): 100.0}

In [39]:
from torch.optim import SGD
from spuco.robust_train import UpSampleERM, DownSampleERM, CustomSampleERM

jtt_train = UpSampleERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    batch_size=64,
    group_partition=jtt_group_partition,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
jtt_train.train()

Epoch 0:   0%|          | 0/1449 [00:00<?, ?batch/s]

Epoch 4: 100%|██████████| 1449/1449 [00:06<00:00, 207.67batch/s, accuracy=67.6470588235294%, loss=0.708]


In [40]:
for name, param in model_fgt.named_parameters():
    if param.requires_grad:
        # print(name, param.data)
        if not torch.isclose(param.data, model.state_dict()[name], atol=1e-4).all():
            print("not equal")

not equal
not equal
not equal
not equal
not equal
not equal


In [42]:
# print parameters in model_jtt
for name, param in model_jtt.named_parameters():
    if param.requires_grad:
        print(name, param.data)

backbone.fc1.weight tensor([[-0.0008,  0.0104, -0.0176,  ..., -0.0132, -0.0008,  0.0031],
        [ 0.0051, -0.0231, -0.0031,  ..., -0.0235,  0.0153,  0.0032],
        [-0.0161,  0.0150, -0.0067,  ..., -0.0169, -0.0055, -0.0120],
        ...,
        [-0.0006,  0.0025,  0.0053,  ..., -0.0176,  0.0083, -0.0107],
        [-0.0123, -0.0154, -0.0055,  ..., -0.0005, -0.0044, -0.0153],
        [ 0.0098,  0.0015,  0.0086,  ...,  0.0033,  0.0145,  0.0052]],
       device='cuda:0')
backbone.fc1.bias tensor([ 1.2130e-02, -9.7771e-03, -9.9735e-03, -1.1502e-02,  6.6799e-03,
        -6.6680e-04, -2.2222e-02, -8.7270e-03, -1.5164e-02,  9.9173e-03,
        -5.5212e-03, -1.1228e-02, -1.6626e-02, -1.2177e-02,  1.9945e-02,
        -8.5539e-03,  1.3544e-02,  2.3331e-02, -1.8100e-02, -1.8582e-02,
        -1.8714e-02,  1.5472e-02,  1.6428e-04,  1.1836e-02, -8.8285e-03,
         7.9330e-03, -1.7030e-02,  8.7986e-03, -2.1723e-02,  1.7956e-02,
        -1.3795e-02,  1.7609e-02, -6.8806e-03,  5.0965e-03, -2.143

In [41]:
# print parameters in model_jtt
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

backbone.fc1.weight tensor([[-0.0024,  0.0088, -0.0192,  ..., -0.0132, -0.0008,  0.0031],
        [ 0.0039, -0.0244, -0.0044,  ..., -0.0233,  0.0155,  0.0034],
        [-0.0161,  0.0150, -0.0067,  ..., -0.0169, -0.0055, -0.0120],
        ...,
        [-0.0006,  0.0025,  0.0053,  ..., -0.0176,  0.0083, -0.0107],
        [-0.0167, -0.0198, -0.0099,  ..., -0.0005, -0.0045, -0.0154],
        [ 0.0059, -0.0024,  0.0048,  ..., -0.0027,  0.0085, -0.0007]],
       device='cuda:0')
backbone.fc1.bias tensor([ 1.0515e-02, -1.1149e-02, -9.9735e-03, -1.7382e-02,  6.6526e-03,
        -7.4012e-03, -2.4037e-02, -8.7270e-03, -2.4446e-02, -5.2563e-03,
        -8.2215e-03, -1.1228e-02, -2.1028e-02, -1.3505e-02,  1.7776e-02,
        -1.0275e-02,  1.7097e-01,  2.1493e-02, -2.4411e-02, -2.3754e-02,
        -1.8670e-02,  1.5472e-02,  5.1165e-05,  1.0639e-02, -8.8285e-03,
        -5.3033e-03, -2.0649e-02,  8.6919e-03, -3.7594e-02,  3.1319e-03,
        -1.7409e-02,  1.7609e-02, -6.8806e-03,  4.8646e-03, -2.525

In [28]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model_jtt,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   0%|          | 0/25 [00:00<?, ?it/s]

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:08,  2.70it/s]

Group (0, 0) Accuracy: 99.76359338061465


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:08,  2.75it/s]

Group (0, 1) Accuracy: 53.90070921985816


...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:08<00:00,  2.72it/s]

Group (4, 3) Accuracy: 24.494949494949495


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:09<00:00,  2.71it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 99.76359338061465,
 (0, 1): 53.90070921985816,
 (0, 2): 66.19385342789599,
 (0, 3): 78.01418439716312,
 (0, 4): 59.1016548463357,
 (1, 0): 58.924205378973106,
 (1, 1): 100.0,
 (1, 2): 23.529411764705884,
 (1, 3): 34.068627450980394,
 (1, 4): 7.598039215686274,
 (2, 0): 33.86666666666667,
 (2, 1): 30.666666666666668,
 (2, 2): 99.46666666666667,
 (2, 3): 9.066666666666666,
 (2, 4): 0.0,
 (3, 0): 62.311557788944725,
 (3, 1): 22.670025188916878,
 (3, 2): 22.418136020151135,
 (3, 3): 99.24433249370277,
 (3, 4): 14.6095717884131,
 (4, 0): 65.4911838790932,
 (4, 1): 7.5566750629722925,
 (4, 2): 0.0,
 (4, 3): 24.494949494949495,
 (4, 4): 100.0}

In [21]:

for group, members in jtt_group_partition.items():
    print(group, len(members))

(0, 0) 46353
(0, 1) 1651


In [20]:
fgt_group_partition.keys()

for group, members in fgt_group_partition.items():
    print(group, len(members))

(0, 0) 45672
(0, 1) 2332


In [24]:
from torch.optim import SGD
from spuco.robust_train import UpSampleERM, DownSampleERM, CustomSampleERM

fgt_train = UpSampleERM(
    model=model_fgt,
    num_epochs=5,
    trainset=trainset,
    batch_size=64,
    group_partition=fgt_group_partition,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=False),
    device=device,
    verbose=True
)
fgt_train.train()

Epoch 0:   0%|          | 0/1428 [00:00<?, ?batch/s]

Epoch 4: 100%|██████████| 1428/1428 [00:06<00:00, 220.72batch/s, accuracy=50.0%, loss=1.38]    


In [25]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model_fgt,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   0%|          | 0/25 [00:00<?, ?it/s]

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:08,  2.87it/s]

Group (0, 0) Accuracy: 99.76359338061465


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:08,  2.87it/s]

Group (0, 1) Accuracy: 53.90070921985816


...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:08<00:00,  2.72it/s]

Group (4, 3) Accuracy: 24.494949494949495


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:09<00:00,  2.72it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 99.76359338061465,
 (0, 1): 53.90070921985816,
 (0, 2): 66.19385342789599,
 (0, 3): 78.01418439716312,
 (0, 4): 59.1016548463357,
 (1, 0): 58.924205378973106,
 (1, 1): 100.0,
 (1, 2): 23.529411764705884,
 (1, 3): 34.068627450980394,
 (1, 4): 7.598039215686274,
 (2, 0): 33.86666666666667,
 (2, 1): 30.666666666666668,
 (2, 2): 99.46666666666667,
 (2, 3): 9.066666666666666,
 (2, 4): 0.0,
 (3, 0): 62.311557788944725,
 (3, 1): 22.670025188916878,
 (3, 2): 22.418136020151135,
 (3, 3): 99.24433249370277,
 (3, 4): 14.6095717884131,
 (4, 0): 65.4911838790932,
 (4, 1): 7.5566750629722925,
 (4, 2): 0.0,
 (4, 3): 24.494949494949495,
 (4, 4): 100.0}

In [49]:
evaluator.average_accuracy

95.97156158648662